# Adverse Event to OMOP

https://docs.google.com/spreadsheets/d/1Vw1Dr2K4oG__cDQTutGaJhZvGUvQTLwc4qWreP6qMSs/edit?gid=507968289#gid=507968289

We have changed the mapping: now an adverse event = observation

Is this right: observation_source_concept_id= the code for the duraton we have in the datamodel?

We will use as concept_id the value from Adverse event type (CTCAE Term), since:

![First image](images/AdverseEvent1.png)

It gives us grading information, does it give us "ADVERSE REACTION" too?

Adverse Reaction = 

In [ ]:
import mysql.connector
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta

In [ ]:
df_Adverse_Event_IDEA4RC = pd.read_csv("./IDEA4RC-data/adverseEventIDEA4RC.csv")
df_Adverse_Event_IDEA4RC.head(5)

In [ ]:
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

cur = conn.cursor()
config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn2 = mysql.connector.connect(**config)
curIDEA = conn2.cursor()

### Adverse Event to Observation

observation_concept_id=4105886
observation type concept id????

In [ ]:
sql = """
    INSERT INTO omopcdm.observation (person_id, observation_concept_id, observation_date,observation_type_concept_id,observation_event_id,obs_event_field_concept_id,observation_source_concept_id)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
"""
#Episode concept id=vocabulary from event type

query= """
    SELECT c.patient 
    FROM cancer_episode c
    WHERE EXISTS (
        SELECT 1 
        FROM EpisodeEvent e
        WHERE e.cancerEpisode = c.id
        AND e.id = %s
    )
    LIMIT 1

"""
queryDate="""
    SELECT episode_start_date
    FROM cancer_episode
    WHERE id = %s
"""
sqlGetEpisode = """
    SELECT c.id 
    FROM episode c
    WHERE c.person_id = %s
    AND c.episode_start_date = %s
    LIMIT 1
"""
observation_type_concept_id=32879

df_tables=df_Adverse_Event_IDEA4RC
for idx, row in df_tables.iterrows():
    curIDEA.execute(query, (row['Episode Event reference']))
    res=curIDEA.fetchone()
    person_id=res
    curIDEA.execute(queryDate, (row['Episode Event reference']))
    res=curIDEA.fetchone()
    observation_date=res
    cur.execute(sqlGetEpisode, (person_id, observation_date))
    episode_id=cur.fetchone()
    observation_concept_id=row['Adverse event type (CTCAE Term)']
    observation_source_concept_id=row['Adverse Event Duration']
    cur.execute(sql, (person_id, observation_concept_id, observation_date, observation_type_concept_id, episode_id, 1147127,observation_source_concept_id))
    conn.commit()